# Classes

In [1]:
# Advanced Planning and Scheduling Demo

# product
class Product:
    def __init__(self, name, demand, priority, production_time, lb_time, ub_time):
        # product attributes
        self.name = name # name/label of the product
        self.demand = demand # demand of the product (quantity required by customers)
        self.min_demand_size = 0 # minimum demand size of the package
        self.priority = priority # priority of the product, higher priority products are produced first
        self.production_time = production_time # time it takes to produce the product
        self.lb_time = lb_time # lower bound time that the product should be produced before
        self.ub_time = ub_time # upper bound time that the product should be produced after
        
        # required resources
        self.required_resources = []
        
    def __str__(self):
        return self.name

#TODO: add resource classes attributes
# resource
class machine:
    def __init__(self, name, capacity, setup_time):
        # resource attributes
        self.name = name # name/label of the resource
        self.capacity = capacity # capacity of the resource
        self.setup_time = setup_time # time it takes to setup the resource
        
        self.able_to_produce = []
        
    def __str__(self):
        return self.name

class worker:
    def __init__(self, name, capacity, setup_time):
        # resource attributes
        self.name = name # name/label of the resource
        self.capacity = capacity # capacity of the resource
        self.setup_time = setup_time # time it takes to setup the resource
        
        self.able_to_produce = []
        
    def __str__(self):
        return self.name



In [2]:
# Create products
product_A = Product('A', 100, 1, 10, 0, 100)
product_B = Product('B', 200, 2, 15, 0, 100)

# Create machine
machine_x = machine('x', 1, 5)

# Create worker
worker_1 = worker('Worker 1', 1, 0)

# Assign products to machine
machine_x.able_to_produce = [product_A, product_B]

# Assign products to worker
worker_1.able_to_produce = [product_A, product_B]

# Mixed Integer Programming for Production Scheduling

### minimize the total cost( $TC$ )

$$ TC = t_c + v_c + s_c + l_c $$
- $t_c$ : sum of time cost that needed to complete all the products
- $v_c$ : sum of vialation cost tat needed
- $s_c$ : short term cost: sum of cost that needed for workers/resources.
- $l_c$ : long term cost: sum of hardware cost that needed, fixed cost.

### via the following constraints of products and workers:
- each product has a deadline, and the deadline is the time that the product should be completed.
  - $\leq$ deadline fine, otherwise violation cost.
  - total_products $\geq$ demand $-$ violations
  - violations $\leq$ allowed_violations (can also be per time period/ total)

- each worker can only work on one machine at a time:
  - $ \sum_{i=1}^{n} x_{i,j} \leq 1 $ $\forall j$ in $[1, m]$ $\textbf{Binary}$

- each machine can only work on one product at a time:
  - $ \sum_{j=1}^{m} x_{i,j} \leq 1 $ $\forall i$ in $[1, n]$ $\textbf{Binary}$

- machine change to produce a new product need a context switch time:
  - HOW TO IMPLEMENT THIS?
  - using a binary variable to indicate the context switch time?
  - $ x_{i,j} + x_{i+1,j} \leq 1 $ $\forall i$ in $[1, n-1]$, $\forall j$ in $[1, m]$ $\textbf{Binary}$ ??? TODO ???

- each worker has a maximum working time:
  - $ worker_{i} \leq max_{worker_{i}} $ $\forall i$ in $[1, n]$

### via the following constraints of supplies and demands:
- each product need enough supplies to begin with
  - $ \sum_{j=1}^{m} x_{i,j} \geq min_{batch_{i}} $ $\forall i$ in $[1, a]$
- each product/supply has cost to stay in the warehouse
  - add the cost of warehouse in short term cost
- some product has minimum size of batch to produce
  - the product need to stay in the warehouse before it can be shipped
      ####  IGNORE THIS FOR NOW
  - $\sum_{j=1}^{m} x_{i,j} \geq min_{batch_{i}}$ $\forall i$ in $[1, a]$


In [7]:
import pulp

# Define the parameters (example values)
products = [1, 2]  # List of products
workers = [1, 2]   # List of workers
machines = [1, 2]  # List of machines
time_periods = [1, 2, 3, 4]  # List of time periods

C = { (1, 1, 1, 1): 10, (1, 1, 1, 2): 12, (1, 1, 1, 3): 14, (1, 1, 1, 4): 16,
      (1, 1, 2, 1): 18, (1, 1, 2, 2): 20, (1, 1, 2, 3): 22, (1, 1, 2, 4): 24,
      # Add more cost values for other combinations
      (1, 2, 1, 1): 10, (1, 2, 1, 2): 12, (1, 2, 1, 3): 14, (1, 2, 1, 4): 16,
      (1, 2, 2, 1): 18, (1, 2, 2, 2): 20, (1, 2, 2, 3): 22, (1, 2, 2, 4): 24,
      (2, 1, 2, 4): 30, (2, 1, 2, 3): 28, (2, 1, 2, 2): 26, (2, 1, 2, 1): 24,
      (2, 1, 1, 4): 22, (2, 1, 1, 3): 20, (2, 1, 1, 2): 18, (2, 1, 1, 1): 16,
      (2, 2, 2, 4): 30, (2, 2, 2, 3): 28, (2, 2, 2, 2): 26, (2, 2, 2, 1): 24,
      (2, 2, 1, 4): 22, (2, 2, 1, 3): 20, (2, 2, 1, 2): 18, (2, 2, 1, 1): 16
    }
D = { 1: 3, 2: 4 }  # Deadlines for products
W = { 1: 2, 2: 3 }  # Maximum working times for workers
S = { 1: 1, 2: 2 }  # Context switch times for products
P = { 1: 1, 2: 2 }  # Minimum batch sizes for products
H = { 1: 5, 2: 6 }  # Storage costs per unit time
R = { 1: 1, 2: 1 }  # Supply requirements for products

# Create the problem
prob = pulp.LpProblem("Production_Scheduling", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts("x", (products, workers, machines, time_periods), cat='Binary')
y = pulp.LpVariable.dicts("y", (products, time_periods), cat='Binary')
z = pulp.LpVariable.dicts("z", (products, time_periods), cat='Binary')

# Objective function
prob += pulp.lpSum(C[i, j, k, t] * x[i][j][k][t] for i in products for j in workers for k in machines for t in time_periods) + \
        pulp.lpSum(H[i] * y[i][t] for i in products for t in time_periods)

# Constraints

# Deadline constraints
for i in products:
    prob += pulp.lpSum(x[i][j][k][t] for j in workers for k in machines for t in time_periods if t <= D[i]) >= 1

# Single machine constraint
for t in time_periods:
    for j in workers:
        prob += pulp.lpSum(x[i][j][k][t] for i in products for k in machines) <= 1

# Single product on machine constraint
for t in time_periods:
    for k in machines:
        prob += pulp.lpSum(x[i][j][k][t] for i in products for j in workers) <= 1

# Context switch constraint
for i in products:
    for t in time_periods:
        if t > 1:
            prob += pulp.lpSum(x[i][j][k][t] for j in workers for k in machines) - \
                    pulp.lpSum(x[i][j][k][t-S[i]] for j in workers for k in machines) <= 0

# Maximum working time constraint
for j in workers:
    prob += pulp.lpSum(x[i][j][k][t] for i in products for k in machines for t in time_periods) <= W[j]

# Supply availability constraint
for i in products:
    for t in time_periods:
        prob += pulp.lpSum(x[i][j][k][t] for j in workers for k in machines) <= R[i]

# Warehouse cost constraint
for i in products:
    for t in time_periods:
        prob += y[i][t] >= pulp.lpSum(x[i][j][k][t] for j in workers for k in machines)

# Minimum batch size constraint
for i in products:
    for t in time_periods:
        prob += pulp.lpSum(x[i][j][k][t] for j in workers for k in machines) >= P[i] * z[i][t]

# Solve the problem
prob.solve()

# Output the results
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

print(f"Total cost = {pulp.value(prob.objective)}")


ModuleNotFoundError: No module named 'pulp'

### indices:
- $i$ : product index: ith product
- $j$ : worker index: jth worker
- $k$ : machine index: kth machine
- $l$ : supply index: lth supply (IGNORE FOR NOW)
- $t$ : time index: tth time period

### variables:
- $x_{i,j,k,t}$ : binary variable, 1 if product i is produced by worker j on machine k at time t, 0 otherwise
- $y_{i,t}$ : binary variable, 1 if product i is in the warehouse at time t, 0 otherwise

  - $s_{l,t}$ : Inventory of supply l at time t
  - $z_{i,t}$ : binary variable, 1 if product i is shipped at time t, 0 otherwise
  - $w_{j,t}$ : binary variable, 1 if worker j is working at time t, 0 otherwise
  - $m_{k,t}$ : binary variable, 1 if machine k is working at time t, 0 otherwise


In [6]:
import pulp

def solve_production_schedule():
    # Create a problem instance
    prob = pulp.LpProblem("Production_Schedule", pulp.LpMinimize)

    # Decision variables
    xA1 = pulp.LpVariable('xA1', lowBound=0, cat='Integer')
    xA2 = pulp.LpVariable('xA2', lowBound=0, cat='Integer')
    xB1 = pulp.LpVariable('xB1', lowBound=0, cat='Integer')
    xB2 = pulp.LpVariable('xB2', lowBound=0, cat='Integer')
    xC1 = pulp.LpVariable('xC1', lowBound=0, cat='Integer')
    xC2 = pulp.LpVariable('xC2', lowBound=0, cat='Integer')

    dA = pulp.LpVariable('dA', lowBound=0, cat='Integer')
    dB = pulp.LpVariable('dB', lowBound=0, cat='Integer')
    dC = pulp.LpVariable('dC', lowBound=0, cat='Integer')

    # Parameters
    pA, pB, pC = 15, 10, 20
    qA, qB, qC = 10, 7, 12
    hA, hB, hC = 1, 3, 2
    iA, iB, iC = 1, 2, 2
    DA, DB, DC = 5, 3, 7
    CA, CB, CC = 1000, 500, 2000

    # Objective function: Minimize penalty cost
    prob += CA * dA + CB * dB + CC * dC, "TotalPenaltyCost"

    # Constraints
    # Production time constraints
    prob += pA * xA1 + qA * xA2 <= 24 * DA, "ProductionA"
    prob += pB * xB1 + qB * xB2 <= 24 * DB, "ProductionB"
    prob += pC * xC1 + qC * xC2 <= 24 * DC, "ProductionC"

    # Demand satisfaction constraints
    prob += xA1 + xA2 >= 5, "DemandA"
    prob += xB1 + xB2 >= 3, "DemandB"
    prob += xC1 + xC2 >= 7, "DemandC"

    # Delay constraints
    prob += dA >= (pA * xA1 + qA * xA2 - 24 * DA) / 24, "DelayA"
    prob += dB >= (pB * xB1 + qB * xB2 - 24 * DB) / 24, "DelayB"
    prob += dC >= (pC * xC1 + qC * xC2 - 24 * DC) / 24, "DelayC"

    # Solve the problem
    prob.solve()

    # Display the results
    print("Optimization results:")
    for v in prob.variables():
        print(f'{v.name}: {v.varValue}')
    print(f'Optimal cost: {pulp.value(prob.objective)}')


ModuleNotFoundError: No module named 'pulp'